# Mount drive

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [ ]:
!pip install jupyter_dash
!pip install --upgrade plotly
!pip install lime
!pip install shap

In [18]:
import os
import datetime
import time
import timeit

import pandas as pd
import numpy as np

import io
import base64

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline


#dropdown
import ipywidgets as widgets
import IPython
from IPython.display import Image
from IPython.core.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual

# deep learning
import tensorflow as tf
from tensorflow import keras
from keras.applications import inception_v3 as inc_net
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
from skimage.segmentation import mark_boundaries
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# dashboard
import dash 
from jupyter_dash import JupyterDash 
import dash_core_components as dcc 
import dash_html_components as html 
import plotly.express as px
from dash.dependencies import Input, Output

# xai
import shap

%load_ext autoreload
%autoreload 2
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [20]:
# vis
import plotly
plotly.offline.init_notebook_mode()

Output hidden; open in https://colab.research.google.com to view.

In [21]:
folder_deep_weeds = 'drive/MyDrive/Industrial_Applications_of_AI/Primary_Sector/DeepWeeds'
data_folder = 'drive/MyDrive/Industrial_Applications_of_AI/Primary_Sector/DeepWeeds/data'
models_folder = 'drive/MyDrive/Industrial_Applications_of_AI/Primary_Sector/DeepWeeds/models'
test_folder = 'drive/MyDrive/Industrial_Applications_of_AI/Primary_Sector/DeepWeeds/data/test_oneClass2'
test_folder_lime = 'drive/MyDrive/Industrial_Applications_of_AI/Primary_Sector/DeepWeeds/data/test_lime'

test_folder_lime_test = 'drive/MyDrive/Industrial_Applications_of_AI/Primary_Sector/DeepWeeds/data/test_lime/test'

CLASSES = [
 'Chinee apple',
 'Lantana',
 'Negative',
 'Parkinsonia',
 'Parthenium',
 'Prickly acacia',
 'Rubber vine',
 'Siam weed',
 'Snake weed']

In [22]:
def loadModelAndResuls(directory):
  model = tf.keras.models.load_model(os.path.join(models_folder,directory, 'model.h5'))
  results_combined = pd.read_csv(os.path.join(models_folder,directory, 'predictions.csv'))
  return model, results_combined

In [23]:
selectable_models = [] # start with chain models ->

for dir in os.listdir(models_folder):
  datetim_string = str(dir).split("_")
  datetim_string = str(datetim_string[0]) +"__" + str(datetim_string[2])
  datetime_object = datetime.datetime.strptime(datetim_string,'%Y-%m-%d__%H-%M-%S')

  threshold =  datetime.datetime.strptime("2022-03-24__18-20-00",'%Y-%m-%d__%H-%M-%S') # fix all chain experiments
  if datetime_object > threshold:
    selectable_models.append(dir)

# Load the model

In [24]:
selectable_models

['2022-03-24__21-15-28_Chain_1_MobileNetV2_full_dataset_very_low_lr',
 '2022-03-24__22-40-51_Chain_1_MobileNetV2_full_dataset_low_lr',
 '2022-03-25__00-14-01_Chain_1_MobileNetV2_full_dataset__medium_lr',
 '2022-03-25__01-02-20_Chain_1_MobileNetV2_full_dataset_high_lr',
 '2022-03-25__02-32-40_Chain_1_MobileNetV2_full_dataset_very_very__low_lr',
 '2022-03-25__14-41-44_Test_run_for_chain']

In [25]:
# select the model
selected_model_folder = '2022-03-25__00-14-01_Chain_1_MobileNetV2_full_dataset__medium_lr'

In [26]:
# load the model
model, results_combined = loadModelAndResuls(selected_model_folder)

# Lime

Use Lime
https://towardsdatascience.com/idea-behind-lime-and-shap-b603d35d34eb

Main paper: https://www.kdd.org/kdd2016/papers/files/rfp0573-ribeiroA.pdf / https://proceedings.neurips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf

https://github.com/marcotcr/lime/blob/master/doc/notebooks/Tutorial%20-%20Image%20Classification%20Keras.ipynb

# Own model

In [27]:
os.listdir(test_folder_lime_test)

['20160928-141421-0.jpg', '20171109-070740-1.jpg', 'cat-dog.jpg']

In [28]:
image1_path = os.path.join(test_folder_lime_test,'20160928-141421-0.jpg')
image2_path = os.path.join(test_folder_lime_test,'20171109-070740-1.jpg')
image3_path = os.path.join(test_folder_lime_test,'cat-dog.jpg')

# XAI with Lime

In [29]:
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(256, 256))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        # x = inc_net.preprocess_input(x)
        x = x/255 #right preprocessing
        out.append(x)
    return np.vstack(out)


# XAI with Shap

In [30]:
def load_Image_no_transform(path_img):
  img = load_img(path_img, target_size=(256, 256))
  img_array = image.img_to_array(img)
  img_batch = np.expand_dims(img_array, axis=0)
  img_batch = img_batch/255
  return img_batch


In [31]:
def f_own(x):
  return model.predict(x)

# Create Interactive builder

In [ ]:
pd.options.plotting.backend = "plotly"


app = JupyterDash(__name__)
app.layout = html.Div([

    dcc.Slider(100, 1000, 100 ,
               id='Shap_slider'
    ),
    dcc.Loading(
                    id="loading-2",
                    children=[html.Img(id='Lime_graph')],
                    type="circle",
                )

    
])



@app.callback(
    dash.dependencies.Output('Lime_graph', 'src'), # src attribute
    [dash.dependencies.Input('Shap_slider', 'value')],
     prevent_initial_call=True
)
def update_figure(n_points):

    # create lime image
    




    #create some matplotlib graph
    x = np.random.rand(n_points)
    y = np.random.rand(n_points)
    buf = io.BytesIO() # in-memory files
    plt.scatter(x, y)
    plt.savefig(buf, format = "png") # save to the above file object
    plt.close()
    data = base64.b64encode(buf.getbuffer()).decode("utf8") # encode to html elements
    return "data:image/png;base64,{}".format(data)

In [ ]:
app.run_server(mode='inline', port = 8093, dev_tools_ui=True,# debug=True,
              dev_tools_hot_reload =True, threaded=True)

# show Lime images

```
`# This is formatted as code`
```



In [ ]:
pd.options.plotting.backend = "plotly"


# plotly figure
# fig = df.plot(template = 'plotly_dark')

app = JupyterDash(__name__)
app.layout = html.Div([
    # dcc.Dropdown(
    #             results_combined['Filename'],
    #             'Select Image',
    #             id='image',
                
    #         ),
    
    
    dcc.Slider(100, 1000, 100 ,
               id='Shap_slider'
    ),
    # dcc.Slider(100, 1000, 100,
              
    #            id='Lime_slider'
    # ),
    # dcc.Graph(id='Image_graph'),
    # dcc.Graph(id='Prediction_graph'),
    # html.Img(id='Shap_graph'),
    dcc.Loading(
                  id="loading-1",
                  children=[html.Img(id='Shap_graph')],
                  type="circle",
    ),
    dcc.Loading(
                  id="loading-2",
                  children=[html.Img(id='Lime_graph')],
                  type="circle",
    )

    
])

# Define callback to update graph
# @app.callback(
#     [
#     Output('Shap_graph', 'src'),
#     #Output('graph2', 'figure')
#     ],
#     [
#     Input('image', 'value'),
#     Input('Shap_slider', 'value'),
#     Input('Lime_slider', 'value'),
#     ],
#     prevent_initial_call=True
# )
# def streamFig(filename, Shap_slider_value ,Lime_slider_value ):
#     # img = io.imread('{}/{}'.format(test_folder,filename))
#     # fig1 = px.imshow(img)

#     # #query the data
#     # results_combined_row = results_combined_new[results_combined_new['Filename']==filename]
#     # # print(filename)
#     # # show the figure

#     # # create a graph for the predictions
#     # #mark the actual class
#     # colors = ['#636EFA',] * 9
#     # if int(results_combined_row[['Actual_num']].iloc[0,0]) == 8:
#     #   colors[2] = 'crimson'
#     # elif int(results_combined_row[['Actual_num']].iloc[0,0]) > 1:
#     #   colors[int(results_combined_row[['Actual_num']].iloc[0,0])+1] = 'crimson'
#     # else:
#     #   colors[int(results_combined_row[['Actual_num']].iloc[0,0])] = 'crimson'
      
    
#     # fig2 = go.Figure([go.Bar(
#     #                     x=CLASSES, 
#     #                     y=np.array(results_combined_row.iloc[: , 3:-1 ])[0],
#     #                     marker_color=colors
#     #                     )],
                        
#     #                 layout_title_text="Predicted: {} <br>Actual: {} (red) <br>Test Image: {}".format(results_combined_row.iloc[0,1],
#     #                                                                                     results_combined_row.iloc[0,12],filename),
                    
#     #                 )

#     #create some matplotlib graph
#     n_points = 10
#     x = np.random.rand(Shap_slider_value)
#     y = np.random.rand(Lime_slider_value)
#     buf = io.BytesIO() # in-memory files
#     plt.scatter(x, y)
#     plt.savefig(buf, format = "png") # save to the above file object
#     plt.close()
#     data = base64.b64encode(buf.getbuffer()).decode("utf8") # encode to html elements

#     return "data:image/png;base64,{}".format(data)


@app.callback(
    [
    #  dash.dependencies.Output('Shap_graph', 'src'), 
     dash.dependencies.Output('Lime_graph', 'src'), 
    ],
    [dash.dependencies.Input('Shap_slider', 'value')],
     prevent_initial_call=True
)
def update_figure(n_points_lime):

    # create lime image
    image2_path = os.path.join(test_folder_lime_test,'20171109-070740-1.jpg')
    images_lime = transform_img_fn([image2_path])
    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(images_lime[0].astype('double'), model, top_labels=5, hide_color=0, num_samples=n_points_lime)


    # # image 1
    # buf = io.BytesIO() # in-memory files
    # temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
    # plt.imshow(mark_boundaries(temp, mask))
    # plt.savefig(buf, format = "png") # save to the above file object
    # plt.close()
    # data = base64.b64encode(buf.getbuffer()).decode("utf8") # encode to html elements


    


    # image 2: combined 
    IMPORTANTCLASS = 0
    buf2 = io.BytesIO() # in-memory files
    amount_of_pic = 20
    
    line = 0
    i_line = 0
    figure, axes = plt.subplots(nrows=(int(amount_of_pic/5))+1, ncols=4,figsize=(15, 15))
    for i in range(0,amount_of_pic):  
        temp, mask = explanation.get_image_and_mask(IMPORTANTCLASS, positive_only=False, num_features=i, hide_rest=False)#hide_rest=False)
    
        if i % 4 == 0 and i != 0:
            line = line+1
            i_line = 0
        
        axes[line,i_line].imshow(mark_boundaries(temp / 1.5 + 0.5, mask))
        axes[line,i_line].set_title('Important Range 1 to {}'.format(i+1))
        i_line = i_line+1

    figure.tight_layout()
    
    plt.savefig(buf2, format = "png") # save to the above file object
    plt.close()
    data2 = base64.b64encode(buf2.getbuffer()).decode("utf8") # encode to html elements



    return "data:image/png;base64,{}".format(data)#, "data:image/png;base64,{}".format(data2)

In [ ]:
app.run_server(mode='inline', port = 8094, dev_tools_ui=True,# debug=True,
              dev_tools_hot_reload =True, threaded=True)

In [32]:
# create lime image
image2_path = os.path.join(test_folder_lime_test,'20171109-070740-1.jpg')
images_lime = transform_img_fn([image2_path])
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(images_lime[0].astype('double'), model, top_labels=5, hide_color=0, num_samples=n_points_lime)


# # image 1
# buf = io.BytesIO() # in-memory files
# temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
# plt.imshow(mark_boundaries(temp, mask))
# plt.savefig(buf, format = "png") # save to the above file object
# plt.close()
# data = base64.b64encode(buf.getbuffer()).decode("utf8") # encode to html elements


# image 2: combined 
IMPORTANTCLASS = 0
buf2 = io.BytesIO() # in-memory files
amount_of_pic = 20

line = 0
i_line = 0
figure, axes = plt.subplots(nrows=(int(amount_of_pic/5))+1, ncols=4,figsize=(15, 15))
for i in range(0,amount_of_pic):  
    temp, mask = explanation.get_image_and_mask(IMPORTANTCLASS, positive_only=False, num_features=i, hide_rest=False)#hide_rest=False)

    if i % 4 == 0 and i != 0:
        line = line+1
        i_line = 0
    
    axes[line,i_line].imshow(mark_boundaries(temp / 1.5 + 0.5, mask))
    axes[line,i_line].set_title('Important Range 1 to {}'.format(i+1))
    i_line = i_line+1

figure.tight_layout()
plt.sh

plt.savefig(buf2, format = "png") # save to the above file object
plt.close()
data2 = base64.b64encode(buf2.getbuffer()).decode("utf8") # encode to html elements

IndentationError: ignored

# Shap image

In [ ]:
pd.options.plotting.backend = "plotly"


# plotly figure
# fig = df.plot(template = 'plotly_dark')

app = JupyterDash(__name__)
app.layout = html.Div([
    # dcc.Dropdown(
    #             results_combined['Filename'],
    #             'Select Image',
    #             id='image',
                
    #         ),
    
    
    dcc.Slider(100, 1000, 100 ,
               id='Shap_slider'
    ),
    # dcc.Slider(100, 1000, 100,
              
    #            id='Lime_slider'
    # ),
    # dcc.Graph(id='Image_graph'),
    # dcc.Graph(id='Prediction_graph'),
    # html.Img(id='Shap_graph'),
    dcc.Loading(
                  id="loading-1",
                  children=[html.Img(id='Shap_graph')],
                  type="circle",
    ),
    dcc.Loading(
                  id="loading-2",
                  children=[html.Img(id='Lime_graph')],
                  type="circle",
    )

    
])

# Define callback to update graph
# @app.callback(
#     [
#     Output('Shap_graph', 'src'),
#     #Output('graph2', 'figure')
#     ],
#     [
#     Input('image', 'value'),
#     Input('Shap_slider', 'value'),
#     Input('Lime_slider', 'value'),
#     ],
#     prevent_initial_call=True
# )
# def streamFig(filename, Shap_slider_value ,Lime_slider_value ):
#     # img = io.imread('{}/{}'.format(test_folder,filename))
#     # fig1 = px.imshow(img)

#     # #query the data
#     # results_combined_row = results_combined_new[results_combined_new['Filename']==filename]
#     # # print(filename)
#     # # show the figure

#     # # create a graph for the predictions
#     # #mark the actual class
#     # colors = ['#636EFA',] * 9
#     # if int(results_combined_row[['Actual_num']].iloc[0,0]) == 8:
#     #   colors[2] = 'crimson'
#     # elif int(results_combined_row[['Actual_num']].iloc[0,0]) > 1:
#     #   colors[int(results_combined_row[['Actual_num']].iloc[0,0])+1] = 'crimson'
#     # else:
#     #   colors[int(results_combined_row[['Actual_num']].iloc[0,0])] = 'crimson'
      
    
#     # fig2 = go.Figure([go.Bar(
#     #                     x=CLASSES, 
#     #                     y=np.array(results_combined_row.iloc[: , 3:-1 ])[0],
#     #                     marker_color=colors
#     #                     )],
                        
#     #                 layout_title_text="Predicted: {} <br>Actual: {} (red) <br>Test Image: {}".format(results_combined_row.iloc[0,1],
#     #                                                                                     results_combined_row.iloc[0,12],filename),
                    
#     #                 )

#     #create some matplotlib graph
#     n_points = 10
#     x = np.random.rand(Shap_slider_value)
#     y = np.random.rand(Lime_slider_value)
#     buf = io.BytesIO() # in-memory files
#     plt.scatter(x, y)
#     plt.savefig(buf, format = "png") # save to the above file object
#     plt.close()
#     data = base64.b64encode(buf.getbuffer()).decode("utf8") # encode to html elements

#     return "data:image/png;base64,{}".format(data)


@app.callback(
    [dash.dependencies.Output('Shap_graph', 'src'), 
     dash.dependencies.Output('Lime_graph', 'src'), 
    ],
    [dash.dependencies.Input('Shap_slider', 'value')],
     prevent_initial_call=True
)
def update_figure(n_points_lime):


    # create shap

    # X = images
    # define a masker that is used to mask out partitions of the input image.
    masker = shap.maskers.Image("inpaint_telea", images[0].shape)

    # create an explainer with model and image masker
    explainer = shap.Explainer(f_own, masker, output_names=CLASSES)

    # create lime image
    image2_path = os.path.join(test_folder_lime_test,'20171109-070740-1.jpg')
    images_lime = transform_img_fn([image2_path])
    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(images_lime[0].astype('double'), model, top_labels=5, hide_color=0, num_samples=n_points_lime)

    buf = io.BytesIO() # in-memory files
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
    plt.imshow(mark_boundaries(temp, mask))
    plt.savefig(buf, format = "png") # save to the above file object
    plt.close()
    data = base64.b64encode(buf.getbuffer()).decode("utf8") # encode to html elements


    IMPORTANTCLASS = 0
    buf2 = io.BytesIO() # in-memory files

    # create plots
    line = 0
    i_line = 0
    figure, axes = plt.subplots(nrows=(int(amount_of_pic/5))+1, ncols=4,figsize=(15, 15))
    for i in range(0,amount_of_pic):  
        temp, mask = explanation.get_image_and_mask(IMPORTANTCLASS, positive_only=False, num_features=i, hide_rest=False)#hide_rest=False)
    
        if i % 4 == 0 and i != 0:
            line = line+1
            i_line = 0
        
        axes[line,i_line].imshow(mark_boundaries(temp / 1.5 + 0.5, mask))
        axes[line,i_line].set_title('Important Range 1 to {}'.format(i+1))
        i_line = i_line+1

    figure.tight_layout()
    
    plt.savefig(buf2, format = "png") # save to the above file object
    plt.close()
    data2 = base64.b64encode(buf2.getbuffer()).decode("utf8") # encode to html elements



    return "data:image/png;base64,{}".format(data), "data:image/png;base64,{}".format(data2)

In [ ]:
app.run_server(mode='inline', port = 8095, dev_tools_ui=True,# debug=True,
              dev_tools_hot_reload =True, threaded=True)

In [ ]:
# create shap

In [ ]:
def create_lime_explained_image (image_path):
  # load the image
  images = transform_img_fn([os.path.join(test_folder,image_path)])

  IMPORTANTCLASS = 0
  # create the explainer
  explainer = lime_image.LimeImageExplainer()
  # magic
  explanation = explainer.explain_instance(images[0].astype('double'), model, top_labels=5, hide_color=0, num_samples=1000)

  # create the combined image
  line = 0
  i_line = 0
  figure, axes = plt.subplots(nrows=(int(amount_of_pic/5))+1, ncols=4,figsize=(15, 15))
  for i in range(0,amount_of_pic):  
      temp, mask = explanation.get_image_and_mask(IMPORTANTCLASS, positive_only=False, num_features=i, hide_rest=False)#hide_rest=False)
  
      if i % 4 == 0 and i != 0:
          line = line+1
          i_line = 0
      
      axes[line,i_line].imshow(mark_boundaries(temp / 1.5 + 0.5, mask))
      axes[line,i_line].set_title('Important Range 1 to {}'.format(i+1))
      i_line = i_line+1

  figure.tight_layout()
  return figure


In [ ]:
pd.options.plotting.backend = "plotly"


# plotly figure
# fig = df.plot(template = 'plotly_dark')

app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Dropdown(
                results_combined['Filename'],
                'Select Image',
                id='image',
                
            ),
    
    dcc.Graph(id='graph1'),
    dcc.Graph(id='graph2'),
    #dcc.Graph(id='graph3'),
])

# Define callback to update graph
@app.callback(
    [
    Output('graph1', 'figure'),
    Output('graph2', 'figure'),
    #Output('graph3', 'figure')
    ],
    [
    Input('image', 'value')
    ],
    prevent_initial_call=True
)
def streamFig(filename):
    img = io.imread('{}/{}'.format(test_folder,filename))
    fig1 = px.imshow(img)

    #query the data
    results_combined_row = results_combined[results_combined['Filename']==filename]
    # print(filename)
    # show the figure

    # create a graph for the predictions
    #mark the actual class
    colors = ['#636EFA',] * 9
    if int(results_combined_row[['Actual_num']].iloc[0,0]) == 8:
      colors[2] = 'crimson'
    elif int(results_combined_row[['Actual_num']].iloc[0,0]) > 1:
      colors[int(results_combined_row[['Actual_num']].iloc[0,0])+1] = 'crimson'
    else:
      colors[int(results_combined_row[['Actual_num']].iloc[0,0])] = 'crimson'
      
    
    fig2 = go.Figure([go.Bar(
                        x=CLASSES, 
                        y=np.array(results_combined_row.iloc[: , 3:-1 ])[0],
                        marker_color=colors
                        )],
                        
                    layout_title_text="Predicted: {} <br>Actual: {} (red) <br>Test Image: {}".format(results_combined_row.iloc[0,1],
                                                                                        results_combined_row.iloc[0,12],filename),
                    
                    )
    # fig3 = create_lime_explained_image(filename)

    return fig1,fig2



In [ ]:
app.run_server(mode='inline', port = 8091, dev_tools_ui=True, debug=True,
              dev_tools_hot_reload =True, threaded=True)

<IPython.core.display.Javascript object>

In [ ]:
def load_Image_no_transform(path_img):
  img = load_img(path_img, target_size=(256, 256))
  img_array = image.img_to_array(img)
  img_batch = np.expand_dims(img_array, axis=0)
  img_batch = img_batch/255
  return img_batch


# Mount drive